In [1]:
!gdown 1OdTQ2yYES10aaUeDvNpfTlF9xI_zirly

Downloading...
From (original): https://drive.google.com/uc?id=1OdTQ2yYES10aaUeDvNpfTlF9xI_zirly
From (redirected): https://drive.google.com/uc?id=1OdTQ2yYES10aaUeDvNpfTlF9xI_zirly&confirm=t&uuid=f9032f5f-974f-4f07-bbb5-beff8747ccd3
To: /content/test_dataset_cls.zip
100% 50.6M/50.6M [00:02<00:00, 25.2MB/s]


In [3]:
!unzip /content/test_dataset_cls.zip

Archive:  /content/test_dataset_cls.zip
replace test/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import ResNet50, VGG16, MobileNetV2
from tensorflow.keras.optimizers import Adam, SGD

# Training config
config = {
    "learning_rate": 0.001,
    "momentum": 0.5,
    "optimizer_type": "adam",
    "batch_size": 128,
    "epochs": 100,
    "loss_function": "categorical_crossentropy",
    "unfreeze": 0.1  # 10%
}

In [11]:
# Data generators
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    'dataset/train', target_size=(100, 100), batch_size=config["batch_size"], class_mode='categorical'
)

valid_generator = datagen.flow_from_directory(
    'dataset/valid', target_size=(100, 100), batch_size=config["batch_size"], class_mode='categorical'
)

test_generator = datagen.flow_from_directory(
    'dataset/test', target_size=(100, 100), batch_size=config["batch_size"], class_mode='categorical', shuffle=False
)

num_classes = train_generator.num_classes

Found 7624 images belonging to 53 classes.
Found 265 images belonging to 53 classes.
Found 265 images belonging to 53 classes.


In [6]:
# Base model
base_model = tf.keras.applications.ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(100, 100, 3)
)
base_model.trainable = False  # Freeze all layers initially

# Custom head
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile
optimizer = Adam(learning_rate=config["learning_rate"])

# Unfreeze last 10% of base_model layers
total_layers = len(base_model.layers)
unfreeze_count = int(total_layers * config["unfreeze"])
for layer in base_model.layers[-unfreeze_count:]:
    layer.trainable = True

# Re-compile and fine-tune
model.compile(optimizer=optimizer, loss=config["loss_function"], metrics=['accuracy'])

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [7]:
# Base model
base_model = tf.keras.applications.MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(100, 100, 3)
)
base_model.trainable = False  # Freeze all layers initially

# Custom head
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile
optimizer = Adam(learning_rate=config["learning_rate"])

# Unfreeze last 10% of base_model layers
total_layers = len(base_model.layers)
unfreeze_count = int(total_layers * config["unfreeze"])
for layer in base_model.layers[-unfreeze_count:]:
    layer.trainable = True

# Re-compile and fine-tune
model.compile(optimizer=optimizer, loss=config["loss_function"], metrics=['accuracy'])

<ipython-input-7-cbc236cfe6ba>:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [10]:
# Base model
base_model = tf.keras.applications.VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(100, 100, 3)
)
base_model.trainable = False  # Freeze all layers initially

# Custom head
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile
optimizer = Adam(learning_rate=config["learning_rate"])

# Unfreeze last 10% of base_model layers
total_layers = len(base_model.layers)
unfreeze_count = int(total_layers * config["unfreeze"])
for layer in base_model.layers[-unfreeze_count:]:
    layer.trainable = True

# Re-compile and fine-tune
model.compile(optimizer=optimizer, loss=config["loss_function"], metrics=['accuracy'])

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [12]:
model.fit(
    train_generator,
    epochs=config["epochs"],
    validation_data=valid_generator
)

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


60/60 ━━━━━━━━━━━━━━━━━━━━ 39s 401ms/step - accuracy: 0.0655 - loss: 3.8051 - val_accuracy: 0.2604 - val_loss: 2.8904
Epoch 2/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 11s 177ms/step - accuracy: 0.2334 - loss: 2.9226 - val_accuracy: 0.3396 - val_loss: 2.4174
Epoch 3/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 11s 175ms/step - accuracy: 0.3163 - loss: 2.5365 - val_accuracy: 0.3774 - val_loss: 2.1310
Epoch 4/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 11s 178ms/step - accuracy: 0.3836 - loss: 2.2816 - val_accuracy: 0.4491 - val_loss: 1.9491
Epoch 5/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 11s 182ms/step - accuracy: 0.4247 - loss: 2.1315 - val_accuracy: 0.5132 - val_loss: 1.8323
Epoch 6/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 11s 185ms/step - accuracy: 0.4552 - loss: 2.0208 - val_accuracy: 0.5396 - val_loss: 1.7275
Epoch 7/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 11s 183ms/step - accuracy: 0.4805 - loss: 1.9281 - val_accuracy: 0.5132 - val_loss: 1.6799
Epoch 8/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 11s 181ms/step - accuracy: 0.4978 - loss: 1.8541 - val_accuracy

In [13]:
# Evaluate on test set
loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy (Keras): {accuracy:.4f}")

# Predict on test data
y_probs = model.predict(test_generator)
y_pred = np.argmax(y_probs, axis=1)
y_true = test_generator.classes

# Compute metrics
acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, average='weighted', zero_division=0)
rec = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

# Print results
print(f"Accuracy Score:  {acc:.4f}")
print(f"Precision Score: {prec:.4f}")
print(f"Recall Score:    {rec:.4f}")
print(f"F1 Score:        {f1:.4f}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 108ms/step - accuracy: 0.7008 - loss: 1.4823
Test Loss: 1.5525
Test Accuracy (Keras): 0.6868
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 481ms/step
Accuracy Score:  0.6868
Precision Score: 0.6946
Recall Score:    0.6868
F1 Score:        0.6742
